In [2]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import calendar

In [ ]:
# ------------------------------
# Load CSV
# ------------------------------
fuel_df = pd.read_csv("csv_data/tbl_carfule.csv")

# Lowercase all text
fuel_df = fuel_df.applymap(lambda x: str(x).lower() if pd.notnull(x) else None)

C:\Users\balot\AppData\Local\Temp\ipykernel_25400\64681004.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fuel_df = fuel_df.applymap(lambda x: str(x).lower() if pd.notnull(x) else None)


In [ ]:
# ------------------------------
# Function to expand month -> daily entries
# ------------------------------
def expand_monthly(row):
    entries = []
    
    # parse month-year
    month_str = row["FuleDT"]
    # start_date = datetime.strptime(month_str, "%B%Y") // Error
    start_date = datetime.strptime(month_str, "%b-%y")
    
    # number of days in that month
    days_in_month = calendar.monthrange(start_date.year, start_date.month)[1]
    
    # daily fuel amount (equal distribution)
    daily_amt = float(row["FuleAmt"]) / days_in_month
    
    for day in range(days_in_month):
        date = start_date + timedelta(days=day)
        entries.append({
            "vehicle_id": row["CarID"],
            "refuel_date": date.strftime("%Y-%m-%d"),
            "fuel_amount": f"{daily_amt:.2f}",
            "created_on": date.strftime("%Y-%m-%d %H:%M:%S"),
            "updated_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
    
    return entries

In [7]:
# ------------------------------
# Expand all rows
# ------------------------------
all_entries = []
for _, row in fuel_df.iterrows():
    all_entries.extend(expand_monthly(row))

final_df = pd.DataFrame(all_entries)

NameError: name 'parse_month_year' is not defined

In [ ]:
# ------------------------------
# Insert into SQLite
# ------------------------------
# Drop duplicates based on refuel_date (and vehicle_id if needed)
# final_df_nodup = final_df.drop_duplicates(subset=["refuel_date"]) // Error
final_df_nodup = final_df.drop_duplicates(subset=["vehicle_id", "refuel_date"])

conn = sqlite3.connect("sqlite_db/guide-mts-data.sqlite3")
final_df_nodup.to_sql("daily_fuel_entries", conn, if_exists="append", index=False)

print("✅ Daily fuel entries inserted successfully")
print(final_df_nodup.head(10))

✅ Daily fuel entries inserted successfully
  vehicle_id refuel_date fuel_amount           created_on           updated_on
0          2  2013-07-01      118.42  2013-07-01 00:00:00  2025-08-24 13:53:38
1          2  2013-07-02      118.42  2013-07-02 00:00:00  2025-08-24 13:53:38
2          2  2013-07-03      118.42  2013-07-03 00:00:00  2025-08-24 13:53:38
3          2  2013-07-04      118.42  2013-07-04 00:00:00  2025-08-24 13:53:38
4          2  2013-07-05      118.42  2013-07-05 00:00:00  2025-08-24 13:53:38
5          2  2013-07-06      118.42  2013-07-06 00:00:00  2025-08-24 13:53:38
6          2  2013-07-07      118.42  2013-07-07 00:00:00  2025-08-24 13:53:38
7          2  2013-07-08      118.42  2013-07-08 00:00:00  2025-08-24 13:53:38
8          2  2013-07-09      118.42  2013-07-09 00:00:00  2025-08-24 13:53:38
9          2  2013-07-10      118.42  2013-07-10 00:00:00  2025-08-24 13:53:38
